## Interactions and dummy variables in R

Let's continue using our ANES data from labs 3 and 5. This time, we'll improve upon our models by incorporating interaction terms and nominal/factor data. 

As always, let's start by installing necessary packages and loading them.o use those commands. 

In [ ]:
# Install the required packages if not already installed 

 install.packages(c('tidyverse', 'haven', 'marginaleffects', 'devtools', 'anesr', 'tidymodels', 'modelsummary'))

# let's load your packages in the R session

library(tidyverse)
library(haven)
library(marginaleffects)
library(devtools)
library(anesr)
library(tidymodels)
library(modelsummary)


Now, load the data and run the recode script from github (this is the same as what we used in previous labs, so you can probably skip the download function).

In [ ]:

# load the dataset from the anesr package
data("timeseries_2020")

# save the url to the R script for recoding and cleaning the data
myurl <- paste0("https://raw.githubusercontent.com/bowendc/510_labs/main/", "lab3_recodes.R")

# download the R script, call the file "lab3recodes.R"
download.file(url = myurl, "lab3recodes.R")

# run the downloaded R script
source("lab3recodes.R")

# recode the variable `sex`
anes20 <- anes20 |> mutate(sex_fct = factor(sex, labels = sex_lbl))

### Regression with nominal/factor data

We can use nominal data with `lm` with the `factor()` function if R doesn't already read the variable as a factor. Let's provide two examples. In the first model below, we've already told R that `sex_fct` has a class of factor:

In [ ]:

m1 <- lm(welfare ~ age + income + sex_fct + pid7, data = anes20)
m2 <- lm(welfare ~ age + income + sex_fct + factor(pid7), data = anes20)

# The output = "jupyter" argument below is used for presentation purposes in
# the juptyer notebook. You would not use that output. Consider "data.frame"
# instead, just let RStudio display the table in the default html in the viewer

# we also set modelsummary() to present p-values as stars in the table and the 
# standard errors below the coefficients (slopes)

modelsummary(list(m1,m2), estimate = "{estimate}{stars}", 
             statistic = "({std.error})", output = "jupyter")

Notice that in the models above, we get are told that the factor category is "women" in `sex_fct`, meaning that respondents who are women have, on average, are 0.032 points lower than men on the welfare spending scale. (This is a five-point ordinal scale from "Increase a lot" to "Decrease a lot"). Women are *more supportive* than men of increasing welfare spending, controlling for the other predictors, but that difference is not statistically distinguishable from 0.  

Model 2 replaces party identification that we treat as ordinal in Model 1 with a series of dummy variables using `factor()`. Each party category dummy shows the difference between the omitted category (Strong Democrat) and the category of the dummy variable. For example, look at the coefficient for `factor(pid7)7`. That number shows the expected change in $\hat{Y}$ as one moves from a Strong Democrat respondent to a Strong Republican respondent, holding the other factors constant. Is it statistically significant? 

### Regression with interaction terms 

Now, let's let the effect of sex vary by partisanship. To do this, we need to fit a model with an interaction term. Mathematically, we are just including a new variable that equals the product of `sex_fct` and `pid7`. The coefficient on this term will show how the coefficients on the constituent terms ($\hat{\beta_{sex_fct}}$ and $$\hat{\beta_{pid7}}$) *change* as the product of the two variables moves up or down.

First, we fit the model and examine the estimates.

In [ ]:
m3 <- lm(welfare ~ age + income + sex_fct*pid7, data = anes20)
tidy(m3)
glance(m3)

Interpreting regression models is difficult! What can we say from this output? 

- `sex_fct`: Women are .10 points higher than men on `welfare`, controlling for the other predictors, *when party ID is 0*. This relationship is statistically significant (p-value < .05). But, `pid7` has a range of 1 to 7, never 0.
- `pid7`: each one-unit movement up the party scale from Strong Dem (1) toward Strong Republican (7), the welfare spending scale goes up (more strongly say "decrease a lot") by .287 points *among men respondents*. Men are those coded 0 on `sex_fct`. Relationship for men is statistically signifcant at pretty much any threshold. 
-  These relationships get more negative when we activate the interaction term. The difference between women and men on predicted welfare attitudes drops by .033 points for each unit increase in `pid7` (because `sex_fctWomen` is a dummy variable, the interaction term simply becomes 1*`pid7`). And each unit increase in `pid7` for women is .033 points smaller than it was for men. 

It helps to graph the **marginal effects* of a key predictor variable of interest. These marginal effects show the estimated relationship between the predictor and $\hat{Y}$ across the values of the other predictor variable interacted with it. Let's plot the marginal effect of `sex_fctWomen` by `pid7` using the `marginaleffects` package.

In [ ]:
# first, calculate the marginal effects using avg_slopes()
# we want the effect of sex_fct
# and pid7 will be on the x axis

mfx <- avg_slopes(m3, variables="sex_fct", by="pid7")

# check the stored data frame
mfx

# now plot the data
# geom_ribbon plots the confidence interval as a shaded region
ggplot(data = mfx, aes(x = pid7)) +
    geom_line(aes(y = estimate)) + 
    geom_ribbon(aes(ymin = conf.low, ymax= conf.high), color = "grey", alpha = .3) + 
    theme_minimal()

Using the `mfx` table, you can see that the marginal effect of `sex_fct` is statistically significant (p-values < .05) and negative when `pid7` is coded as a 5, 6, or 7 (i.e., Republicans). For Republican respondents, sex is a significant predictor of welfare attitudes with women showing less appetite for decreasing welfare spending then men do.